# Machine Learning in Julia

## A neural network in ~10 lines of Julia code

Let's define a fully-connected two layer network.

In [1]:
dense(W, b, σ = identity) = x -> σ.(W * x .+ b)

chain(f...) = foldl(∘, reverse(f))

m = chain(
    dense(randn(5,10), randn(5), tanh),
    dense(randn(2,5), randn(2)))

#62 (generic function with 1 method)

In [2]:
x = rand(10); # some input

In [3]:
m(x)

2-element Array{Float64,1}:
  1.0555215948208532
 -1.1282331856871877

Let's train it!

In [6]:
using Zygote # source-to-source reverse mode AD

dm, = gradient(m) do model
    sum(model(x))
end

((f = (W = [0.9977973541177967 -0.6521435180185351 … 0.9746946274348455 0.49804803007453163; 0.9977973541177967 -0.6521435180185351 … 0.9746946274348455 0.49804803007453163], b = 2-element Fill{Float64}: entries equal to 1.0, σ = nothing), g = (W = [0.0003267528993163552 8.235337950859827e-5 … 0.0003077195212771848 8.780016807789375e-5; -0.8120126092212574 -0.20465612612122927 … -0.7647128209218496 -0.21819192337745505; … ; 0.02150220735087092 0.005419322815325392 … 0.020249702347726273 0.00577775261796336; -0.2569969726791718 -0.06477239916735249 … -0.2420268819893593 -0.06905639534936399], b = [0.0003614959157579772, -0.8983523708331796, 0.04879567395205464, 0.02378849630220229, -0.28432297366009907], σ = nothing)),)

In [7]:
m.f.W

2×5 Array{Float64,2}:
 -0.9582   -1.04853   -0.401626  -0.163496  -0.0476048
  1.04035  -0.514616   3.53206    0.639548  -0.33051

In [8]:
dm.f.W

2×5 Array{Float64,2}:
 0.997797  -0.652144  -0.992176  0.974695  0.498048
 0.997797  -0.652144  -0.992176  0.974695  0.498048

In [9]:
η = 0.01

m.f.W .-= η * dm.f.W # Gradient descent!

2×5 Array{Float64,2}:
 -0.968178  -1.04201   -0.391704  -0.173243  -0.0525853
  1.03037   -0.508094   3.54198    0.629801  -0.335491

## Flux - The ML library that doesn't make you tensor

Web page: https://fluxml.ai/, Examples: [Model zoo](https://github.com/FluxML/model-zoo/)

<img src="https://fluxml.ai/logo.png" width=300>

<img src="flux.png" width=800>

In [11]:
using Flux

In [12]:
m = Chain(
    Dense(10, 5),
    Dense(5, 2),
    softmax # normalize output neurons
)

Chain(Dense(10, 5), Dense(5, 2), softmax)

In [13]:
data, labels = rand(10, 100), fill(0.5, 2, 100); # fake data

In [14]:
loss(x, y) = sum(Flux.mse(m(x), y)) # mean squared error

loss (generic function with 1 method)

In [15]:
opt = Descent(0.01) # or ADAM

Descent(0.01)

In [16]:
Flux.train!(loss, params(m), [(data,labels)], opt)

In [17]:
m(rand(10)) # trained model

2-element Array{Float32,1}:
 0.6105326
 0.38946745

# Example: Machine learning the Ising phase transition

https://juliaphysics.github.io/PhysicsTutorials.jl/tutorials/machine_learning/ml_ising/ml_ising.html